In [8]:
import json

response = {
  "id": "msg_01Nrat83eoPvSqc6DtpTpCuN",
  "type": "message",
  "role": "assistant",
  "model": "claude-sonnet-4-6",
  "content": [
    {
      "type": "thinking",
      "thinking": "The user wants me to rewrite and improve the lab with:\n1. More description\n2. More readable code\n3. Easy to understand for students\n4. Output in Jupyter format using `# %%` cell markers\n\nLet me rewrite this lab making it more educational, with better comments, clearer explanations, and more descriptive markdown cells.",
      "signature": "EuMDCkYICxgCKkBmhGeaFUuwscWu4p/qyCsV3kJQy35zg7KPgUO2CmBzogHpShz7O9a9ddzgmDBupyeFcQ71g/sWbLOjjuzQfVKkEgzq8ITMdr2QWNsyUjQaDMD6678dPfE0WcDrdyIwZdumpXMQboCtp35zrIYZSdmSgW6SOidTU3yoeapdhobUgrblWH9gvCZi8UrQ43QQKsoC+98c3JR1EV0kTKk3TN5D5AZNj1XjXtqSla3ytAJgoEFLC06u/5WF4ZxpSgmjhApOBNJxms+UuZ165lpbvuKLLcmMsxQ1CRy3kJho4b8BFEYKNxsLOX/ghCAOw++XROsyudC0j3kNWH6iO5HHj+J9fDPSh+vX1AzDwreGEAlYRqPZc9sdj8raGWyU1W7f+MOYgo2Dfvd39Hiu/eQFD5stMBOKwvffjOF3vN+r0+ZgXAfIyMQXByuYjq0Y1Vq90SWhn+9yCizr7pUIGt4+DqAJHxoDoN+5QHeQafyuF6RW2soq6HUb1JDnvX1EP31plBy/sxa4H68yorusOHhkxJJfaXU+2TGWqjN8GvjTY8v2dr9omxHIwYFjOfOnkjkUhHEllEL9hN6wK2OpYJKzELMQY2UHG1yDjr2lTW/Fn8KzOQUankgikwR1UcyoGAE="
    },
    {
      "type": "text",
      "text": "# %%\n```python\n# %% [markdown]\n# # 🧪 Lab: Prompt Engineering กับ MLflow\n# ## เรียนรู้ 3 เทคนิค + Reflection Loop + การติดตามผลด้วย MLflow\n#\n# ---\n#\n# ## 🎯 เป้าหมายของ Lab นี้\n#\n# ในการพัฒนา AI Application จริง เราต้องการ:\n# 1. **เทคนิคการเขียน Prompt ที่ดี** → ให้ AI ตอบถูกต้องและมีคุณภาพ\n# 2. **ระบบติดตามผล** → รู้ว่า Prompt เวอร์ชันไหนดีที่สุด\n# 3. **การปรับปรุงอัตโนมัติ** → ให้ AI ช่วยปรับ Prompt เอง\n#\n# Lab นี้จะสอน 3 เทคนิคหลัก + วิธีใช้ MLflow ติดตามทุกอย่าง\n#\n# ---\n#\n# ## 🗺️ แผนผังรวม (Big Picture)\n#\n# ```\n# ┌─────────────────────────────────────────────────────────────────┐\n# │                    3 เทคนิค Prompt Engineering                  │\n# │                                                                 │\n# │  Technique 1: Self-Consistency                                  │\n# │  ┌─────────────────────────────────────────────────────┐        │\n# │  │  ถามคำถามเดิม 5 ครั้ง → เปรียบเทียบ → โหวตเสียงข้างมาก  │        │\n# │  │  Problem → [Path1: 42] [Path2: 42] [Path3: 38] → 42 │        │\n# │  └─────────────────────────────────────────────────────┘        │\n# │                                                                 │\n# │  Technique 2: Prompt Chaining                                   │\n# │  ┌─────────────────────────────────────────────────────┐        │\n# │  │  แบ่งงานเป็นขั้น ส่ง output ต่อกันเป็นทอดๆ          │        │\n# │  │  Review → [Extract] → [Analyze] → [Report]          │        │\n# │  └─────────────────────────────────────────────────────┘        │\n# │                                                                 │\n# │  Technique 3: Self-Critique                                     │\n# │  ┌─────────────────────────────────────────────────────┐        │\n# │  │  ให้ AI ตรวจงานตัวเอง แล้วแก้ไข                    │        │\n# │  │  [Generate Draft] → [Critique] → [Revise] → Final  │        │\n# │  └─────────────────────────────────────────────────────┘        │\n# │                              ↓                                  │\n# │           Reflection Loop: ทดสอบ → วิเคราะห์ → ปรับ Prompt     │\n# │                              ↓                                  │\n# │           MLflow: บันทึก Version, Metrics, Parameters           │\n# └─────────────────────────────────────────────────────────────────┘\n# ```\n\n# %% [markdown]\n# ---\n# ## ⚙️ ส่วนที่ 0: ติดตั้งและตั้งค่า\n#\n# เริ่มต้นด้วยการ import library ที่จำเป็น และตั้งค่าการเชื่อมต่อ\n\n# %%\n# ─────────────────────────────────────────────────────────\n#  Import Libraries\n# ─────────────────────────────────────────────────────────\n\nimport mlflow\nimport mlflow.genai\nimport json\nimport time\nimport re\n\nfrom collections import Counter\nfrom typing import List, Dict, Any\n\nfrom google import genai\nfrom google.genai import types\n\n# ─────────────────────────────────────────────────────────\n#  การตั้งค่าหลัก (แก้ไขตรงนี้)\n# ─────────────────────────────────────────────────────────\n\nAPI_KEY   = \"YOUR_GEMINI_API_KEY\"   # <<<< ใส่ Gemini API Key ของคุณ\nMODEL_ID  = \"gemini-2.0-flash\"      # โมเดลที่ใช้\nMLFLOW_URI = \"http://127.0.0.1:5000\" # ที่อยู่ MLflow Server\n\n# ─────────────────────────────────────────────────────────\n#  สร้าง Client และเชื่อมต่อ MLflow\n# ─────────────────────────────────────────────────────────\n\nclient = genai.Client(api_key=API_KEY)\n\nmlflow.set_tracking_uri(MLFLOW_URI)\nmlflow.set_experiment(\"prompt_engineering_lab\")  # ชื่อ Experiment ใน MLflow\n\nprint(\"=\" * 50)\nprint(f\"  ✅ Gemini Model : {MODEL_ID}\")\nprint(f\"  ✅ MLflow URI   : {mlflow.get_tracking_uri()}\")\nprint(\"=\" * 50)\n\n# %% [markdown]\n# ### 💡 ทำไมต้องใช้ Version Tracker?\n#\n# MLflow เก็บ Prompt เป็น Version (1, 2, 3, ...) แต่**ไม่รองรับ \"latest\"** โดยตรง\n# เราจึงสร้าง dictionary ไว้ติดตาม version ล่าสุดของแต่ละ prompt เอง\n#\n# ```\n# _prompt_versions = {\n#     \"sc_math_solver\": 3,    ← ตอนนี้อยู่ที่ version 3\n#     \"chain_extract\":  1,    ← ยังอยู่ version 1\n#     ...\n# }\n# ```\n\n# %%\n# ─────────────────────────────────────────────────────────\n#  Version Tracker: ติดตาม Version ล่าสุดของแต่ละ Prompt\n# ─────────────────────────────────────────────────────────\n\n_prompt_versions: Dict[str, int] = {}\n\n\ndef track_version(name: str, version: int):\n    \"\"\"\n    บันทึก version ล่าสุดของ prompt ลงใน dictionary\n    ถ้า version ใหม่สูงกว่า version เดิม ให้อัปเดต\n    \"\"\"\n    current_max = _prompt_versions.get(name, 0)\n    _prompt_versions[name] = max(current_max, version)\n\n\ndef get_latest_version(name: str) -> int:\n    \"\"\"\n    ดึง version ล่าสุดของ prompt\n    ถ้ายังไม่เคย track ไว้ คืนค่า 1 เป็น default\n    \"\"\"\n    return _prompt_versions.get(name, 1)\n\n\ndef load_latest_prompt(name: str):\n    \"\"\"\n    โหลด prompt version ล่าสุดจาก MLflow Registry\n    ใช้ format: prompts:/<ชื่อ>/<version>\n    \"\"\"\n    latest_version = get_latest_version(name)\n    prompt_uri = f\"prompts:/{name}/{latest_version}\"\n    return mlflow.genai.load_prompt(prompt_uri)\n\n\n# ─────────────────────────────────────────────────────────\n#  ทดสอบว่า Version Tracker ทำงานได้\n# ─────────────────────────────────────────────────────────\n\ntrack_version(\"test_prompt\", 1)\ntrack_version(\"test_prompt\", 3)\ntrack_version(\"test_prompt\", 2)  # ไม่ควรทับ version 3\n\nassert get_latest_version(\"test_prompt\") == 3, \"Version Tracker มีปัญหา!\"\nprint(\"  ✅ Version Tracker ทำงานถูกต้อง\")\nprint(f\"     track 1, 3, 2 → latest = {get_latest_version('test_prompt')}\")\n\n# %% [markdown]\n# ---\n# ## 🎨 ส่วนที่ 1: Helper Functions สำหรับแสดงผล\n#\n# เราจะสร้างฟังก์ชันช่วยแสดงผลสวยงาม ใน Jupyter Notebook\n# เพื่อให้อ่านและเข้าใจผลลัพธ์ได้ง่ายขึ้น\n\n# %%\n# ─────────────────────────────────────────────────────────\n#  ฟังก์ชันหลัก: เรียก Gemini API\n# ─────────────────────────────────────────────────────────\n\ndef call_gemini(prompt: str,\n                temperature: float = 0.0,\n                max_tokens: int = 2048) -> str:\n    \"\"\"\n    ส่ง prompt ไปให้ Gemini และรับ text กลับมา\n\n    Parameters:\n        prompt      : ข้อความที่จะส่งให้ AI\n        temperature : ความสร้างสรรค์ (0.0 = แม่นยำ, 1.0 = สร้างสรรค์)\n        max_tokens  : จำนวน token สูงสุดของคำตอบ\n\n    Returns:\n        str : คำตอบจาก Gemini เป็น text\n    \"\"\"\n    response = client.models.generate_content(\n        model=MODEL_ID,\n        contents=prompt,\n        config=types.GenerateContentConfig(\n            temperature=temperature,\n            max_output_tokens=max_tokens,\n        ),\n    )\n    return response.text\n\n\n# ─────────────────────────────────────────────────────────\n#  ฟังก์ชันแสดงผล: หัวข้อ (Title)\n# ─────────────────────────────────────────────────────────\n\ndef print_title(text: str, level: int = 1):\n    \"\"\"\n    แสดงหัวข้อแบ่งระดับ\n\n    level 1 → หัวข้อใหญ่ (มีกรอบ ━━━)\n    level 2 → หัวข้อรอง (เส้น ──)\n    level 3 → หัวข้อย่อย (จุด ▸)\n    \"\"\"\n    if level == 1:\n        border = \"━\" * 62\n        print(f\"\\n{border}\")\n        print(f\"  🔷 {text}\")\n        print(f\"{border}\")\n\n    elif level == 2:\n        padding = \"─\" * max(1, 48 - len(text))\n        print(f\"\\n  ── {text} {padding}\")\n\n    else:  # level 3\n        print(f\"\\n    ▸ {text}\")\n\n\n# ─────────────────────────────────────────────────────────\n#  ฟังก์ชันแสดงผล: กล่องข้อความ (Box)\n# ─────────────────────────────────────────────────────────\n\ndef print_box(title: str, content: str,\n              emoji: str = \"📌\", width: int = 78):\n    \"\"\"\n    แสดงข้อความในกล่องสวยงาม พร้อม Word-wrap อัตโนมัติ\n\n    Parameters:\n        title   : ชื่อกล่อง\n        content : เนื้อหาภายในกล่อง\n        emoji   : ไอคอนหัวกล่อง\n        width   : ความกว้างกล่อง (characters)\n    \"\"\"\n    inner_width = width - 4  # พื้นที่ข้อความ (ลบกรอบ 4 ด้าน)\n\n    # วาดกรอบบน\n    print(f\"\\n  ┌{'─' * (width - 2)}┐\")\n    print(f\"  │ {emoji} {title:<{inner_width - 2}}│\")\n    print(f\"  ├{'─' * (width - 2)}┤\")\n    print(f\"  │{' ' * (width - 2)}│\")\n\n    # แสดงเนื้อหาทีละบรรทัด พร้อม Word-wrap\n    for line in content.split(\"\\n\"):\n        line = line.rstrip()\n\n        if not line:\n            # บรรทัดว่าง\n            print(f\"  │{' ' * (width - 2)}│\")\n            continue\n\n        # ตัดบรรทัดยาวให้พอดีกล่อง\n        while len(line) > inner_width:\n            cut_point = line[:inner_width].rfind(\" \")\n            if cut_point <= 0:\n                cut_point = inner_width  # ตัดกลางคำถ้าจำเป็น\n            print(f\"  │ {line[:cut_point]:<{inner_width}} │\")\n            line = line[cut_point:].lstrip()\n\n        print(f\"  │ {line:<{inner_width}} │\")\n\n    # วาดกรอบล่าง\n    print(f\"  │{' ' * (width - 2)}│\")\n    print(f\"  └{'─' * (width - 2)}┘\")\n\n\n# ─────────────────────────────────────────────────────────\n#  ฟังก์ชันแสดงผล: ตาราง (Table)\n# ─────────────────────────────────────────────────────────\n\ndef print_table(headers: List[str],\n                rows: List[List[str]],\n                title: str = \"\"):\n    \"\"\"\n    แสดงตารางสวยงาม พร้อมปรับความกว้างแต่ละคอลัมน์อัตโนมัติ\n\n    Parameters:\n        headers : ชื่อคอลัมน์ ['Col A', 'Col B', ...]\n        rows    : ข้อมูลแต่ละแถว [['a1', 'b1'], ['a2', 'b2'], ...]\n        title   : หัวตาราง (optional)\n    \"\"\"\n    if title:\n        print(f\"\\n  📊 {title}\")\n\n    # คำนวณความกว้างแต่ละคอลัมน์ = max(ความยาว header, ความยาว data)\n    col_widths = [\n        max(\n            len(str(header)),\n            max((len(str(row[i])) for row in rows), default=0)\n        ) + 2\n        for i, header in enumerate(headers)\n    ]\n\n    # สร้าง separator แต่ละคอลัมน์\n    def make_separator(left, mid, right):\n        return f\"  {left}\" + f\"{mid}\".join(\"─\" * (w + 1) for w in col_widths) + f\"─{right}\"\n\n    # วาดหัวตาราง\n    print(make_separator(\"┌\", \"─┬\", \"─┐\"))\n\n    header_line = \"  │\".join(f\" {h:^{w}}\" for h, w in zip(headers, col_widths))\n    print(f\"  │{header_line} │\")\n\n    print(make_separator(\"├\", \"─┼\", \"─┤\"))\n\n    # วาดข้อมูลแต่ละแถว\n    for row in rows:\n        row_line = \"  │\".join(f\" {str(v):^{w}}\" for v, w in zip(row, col_widths))\n        print(f\"  │{row_line} │\")\n\n    print(make_separator(\"└\", \"─┴\", \"─┘\"))\n\n\n# ─────────────────────────────────────────────────────────\n#  ฟังก์ชันแสดงผล: Progress Bar และ Metric\n# ─────────────────────────────────────────────────────────\n\ndef print_progress(current: int, total: int,\n                   label: str = \"\", bar_width: int = 30):\n    \"\"\"\n    แสดง Progress Bar แบบ text\n    ตัวอย่าง: [████████░░░░░░░░░░░░░░░░░░░░░░] 27% Iter 1\n\n    \"\"\"\n    pct = current / total if total > 0 else 0\n    filled = int(bar_width * pct)\n    bar = \"█\" * filled + \"░\" * (bar_width - filled)\n    print(f\"  [{bar}] {pct:.0%}  {label}\")\n\n\ndef print_metric(label: str, value, unit: str = \"\", emoji: str = \"📈\"):\n    \"\"\"\n    แสดง Metric สั้นๆ\n    ตัวอย่าง: 📈 Accuracy: 83.3%\n    \"\"\"\n    if isinstance(value, float):\n        print(f\"    {emoji} {label}: {value:.1f}{unit}\")\n    else:\n        print(f\"    {emoji} {label}: {value}{unit}\")\n\n\ndef print_divider(char: str = \"─\", width: int = 60):\n    \"\"\"แสดงเส้นคั่น\"\"\"\n    print(f\"  {char * width}\")\n\n\n# ─────────────────────────────────────────────────────────\n#  ทดสอบ Helper Functions\n# ─────────────────────────────────────────────────────────\n\nprint_title(\"ทดสอบ Helper Functions\", 1)\nprint_title(\"ระดับ 2: หัวข้อรอง\", 2)\nprint_title(\"ระดับ 3: หัวข้อย่อย\", 3)\n\nprint_box(\n    title   = \"ตัวอย่าง Box\",\n    content = \"บรรทัดที่ 1: Hello World\\nบรรทัดที่ 2: สวัสดีโลก\\n\\nบรรทัดว่าง แล้วต่อ...\",\n    emoji   = \"🎯\",\n)\n\nprint_table(\n    headers = [\"เทคนิค\", \"ความซับซ้อน\", \"ใช้เมื่อ\"],\n    rows    = [\n        [\"Self-Consistency\", \"ปานกลาง\", \"Math / Logic\"],\n        [\"Prompt Chaining\",  \"สูง\",     \"งานหลายขั้น\"],\n        [\"Self-Critique\",    \"สูง\",     \"ต้องการ Accuracy สูง\"],\n    ],\n    title = \"ตัวอย่างตาราง\",\n)\n\nprint_progress(70, 100, \"Example 70%\")\nprint_metric(\"Test Score\", 92.5, \"%\", \"🏆\")\n\n# %%\n# ─────────────────────────────────────────────────────────\n#  ทดสอบการเชื่อมต่อ Gemini API\n# ─────────────────────────────────────────────────────────\n\nprint_title(\"ทดสอบการเชื่อมต่อ Gemini API\")\n\ntest_prompt = \"ตอบสั้นๆ ภายใน 1 ประโยค: MLflow ช่วยอะไรในการพัฒนา AI?\"\ntest_answer = call_gemini(test_prompt, max_tokens=100)\n\nprint_box(\n    title   = \"คำถามทดสอบ\",\n    content = f\"Q: {test_prompt}\\n\\nA: {test_answer}\",\n    emoji   = \"🧪\",\n)\n\n# %% [markdown]\n# ---\n# ## 📘 เทคนิคที่ 1: Self-Consistency (SC)\n#\n# ### หลักการ\n#\n# แทนที่จะถามคำถามครั้งเดียว → ถาม **N ครั้ง** ด้วย temperature สูง\n# (ทำให้ได้ reasoning paths ที่หลากหลาย) → **โหวตเสียงข้างมาก**\n#\n# ### ทำไมถึงได้ผล?\n#\n# - AI อาจเดินทางผิดในบางครั้ง แต่ถ้าถาม 5 ครั้ง ส่วนใหญ่จะได้คำตอบถูก\n# - Majority voting ช่วยกรองข้อผิดพลาดออก\n# - ยิ่ง N มาก ยิ่งแม่นยำ (แต่ใช้เวลาและ token มากขึ้น)\n#\n# ### แผนผัง\n#\n# ```\n#                    ┌─ Path 1 (temp=0.7) ── Answer: 42 ─┐\n#                    │                                     │\n#  Question ─────────┼─ Path 2 (temp=0.7) ── Answer: 42 ─┼──→ โหวต → 42 ✅\n#                    │                                     │    (2/3)\n#                    └─ Path 3 (temp=0.7) ── Answer: 38 ─┘\n# ```\n#\n# ### เหมาะกับงานประเภทไหน?\n#\n# | ✅ เหมาะ | ❌ ไม่เหมาะ |\n# |---------|-----------|\n# | คณิตศาสตร์ | งานสร้างสรรค์ |\n# | Logic / Reasoning | งานที่ต้องการ variety |\n# | คำถามที่มีคำตอบเดียว | ต้องการประหยัด cost |\n\n# %%\n# ─────────────────────────────────────────────────────────\n#  ลงทะเบียน Self-Consistency Prompt v1 ใน MLflow\n# ─────────────────────────────────────────────────────────\n\nprint_title(\"เทคนิคที่ 1: Self-Consistency\")\n\nSC_PROMPT_NAME = \"sc_math_solver\"\n\n# Prompt Version 1: ง่าย ตรงไปตรงมา\nSC_PROMPT_V1 = \"\"\"วิเคราะห์ปัญหาต่อไปนี้และให้คำตอบ\n\nปัญหา: {{ problem }}\n\nแสดงวิธีคิดทีละขั้นตอน แล้วสรุปคำตอบเป็นตัวเลข\"\"\"\n\n# ลงทะเบียนใน MLflow Prompt Registry\nsc_prompt_v1 = mlflow.genai.register_prompt(\n    name=SC_PROMPT_NAME,\n    template=SC_PROMPT_V1,\n    commit_message=\"v1: Basic prompt — minimal instruction\",\n)\n\ntrack_version(SC_PROMPT_NAME, sc_prompt_v1.version)\n\nprint(f\"  ✅ ลงทะเบียนสำเร็จ: '{SC_PROMPT_NAME}' version {sc_prompt_v1.version}\")\nprint_box(\n    title   = f\"SC Prompt v{sc_prompt_v1.version}\",\n    content = SC_PROMPT_V1,\n    emoji   = \"📝\",\n)\n\n# %%\n# ─────────────────────────────────────────────────────────\n#  Self-Consistency Engine: ส่วนสำคัญของเทคนิค\n# ─────────────────────────────────────────────────────────\n\ndef extract_number_from_text(text: str) -> str:\n    \"\"\"\n    สกัดตัวเลขสุดท้ายจาก text\n    เหตุผล: AI มักสรุปคำตอบไว้ตอนท้าย เช่น \"ดังนั้น คำตอบคือ 42\"\n\n    ตัวอย่าง:\n        \"...ดังนั้น คำตอบคือ 1,575 บาท\" → \"1575\"\n        \"แต่ละคนได้ 4 ลูก\"               → \"4\"\n    \"\"\"\n    # หาตัวเลขทั้งหมด (รวม comma และ decimal)\n    all_numbers = re.findall(r\"\\b\\d[\\d,]*(?:\\.\\d+)?\\b\", text)\n\n    if not all_numbers:\n        return \"N/A\"\n\n    # เอาตัวเลขสุดท้าย (มักเป็นคำตอบสุดท้าย) และลบ comma ออก\n    last_number = all_numbers[-1].replace(\",\", \"\")\n    return last_number\n\n\ndef run_self_consistency(template: str,\n                         problem: str,\n                         n_samples: int = 5,\n                         temperature: float = 0.7,\n                         show_details: bool = True) -> Dict:\n    \"\"\"\n    รัน Self-Consistency: สุ่มคำตอบ N ครั้ง → Majority Vote\n\n    Parameters:\n        template     : Prompt template (มี {{ problem }})\n        problem      : โจทย์ที่ต้องการคำตอบ\n        n_samples    : จำนวนครั้งที่ถาม (มากขึ้น = แม่นยำขึ้น แต่ช้าลง)\n        temperature  : ความหลากหลายของคำตอบ (สูง = หลากหลาย)\n        show_details : True = แสดงทุก Path (debug mode)\n                       False = แสดงแค่สรุป (fast mode)\n\n    Returns:\n        dict: {\n            \"final_answer\" : คำตอบที่ชนะการโหวต\n            \"confidence\"   : สัดส่วน (เช่น 0.6 = 3/5)\n            \"answers\"      : list คำตอบทุก path\n            \"distribution\" : การกระจายของคำตอบ\n        }\n    \"\"\"\n    # แทน placeholder ด้วยโจทย์จริง\n    filled_prompt = template.replace(\"{{ problem }}\", problem)\n\n    answers_collected = []\n    full_responses = []\n\n    print_title(f\"รัน {n_samples} Reasoning Paths (temperature={temperature})\", 2)\n\n    # ── รัน N paths ──\n    for path_num in range(1, n_samples + 1):\n\n        # เรียก AI\n        response = call_gemini(filled_prompt, temperature=temperature)\n        full_responses.append(response)\n\n        # สกัดคำตอบตัวเลข\n        extracted_ans = extract_number_from_text(response)\n        answers_collected.append(extracted_ans)\n\n        if show_details:\n            # แสดงผล path เต็มๆ ใน box\n            print_box(\n                title   = f\"Path {path_num}/{n_samples}  →  คำตอบ: {extracted_ans}\",\n                content = response,\n                emoji   = \"🧠\",\n            )\n        else:\n            # แสดงสรุปสั้น (1 บรรทัด)\n            preview = response.replace(\"\\n\", \" \")[:65]\n            status  = \"✅\" if extracted_ans != \"N/A\" else \"⚠️\"\n            print(f\"      {status} Path {path_num}: ans={extracted_ans:>10s}  │  {preview}...\")\n\n        time.sleep(1.0)  # หน่วงเวลาเล็กน้อย ป้องกัน rate limit\n\n    # ── Majority Voting ──\n    # นับเฉพาะคำตอบที่สกัดได้ (ไม่นับ \"N/A\")\n    vote_counter = Counter(a for a in answers_collected if a != \"N/A\")\n\n    if vote_counter:\n        winner_ans, winner_count = vote_counter.most_common(1)[0]\n        confidence = winner_count / n_samples\n    else:\n        winner_ans, confidence = \"N/A\", 0.0\n\n    # ── แสดงผล Vote ──\n    vote_lines = []\n    for ans, count in vote_counter.most_common():\n        filled_bar = \"█\" * (count * 6)\n        empty_bar  = \"░\" * ((n_samples - count) * 6)\n        is_winner  = \"  ← Winner ✅\" if ans == winner_ans else \"\"\n        vote_lines.append(f\"  {ans:>12s} :  {filled_bar}{empty_bar}  ({count}/{n_samples}){is_winner}\")\n\n    vote_lines.append(f\"\\n  🏆 Final Answer : {winner_ans}\")\n    vote_lines.append(f\"  📊 Confidence   : {confidence:.0%}  ({int(confidence * n_samples)}/{n_samples} เห็นด้วย)\")\n\n    print_box(\n        title   = \"ผลการโหวต (Majority Vote)\",\n        content = \"\\n\".join(vote_lines),\n        emoji   = \"🗳️\",\n    )\n\n    return {\n        \"final_answer\" : winner_ans,\n        \"confidence\"   : confidence,\n        \"answers\"      : answers_collected,\n        \"distribution\" : dict(vote_counter),\n        \"all_responses\": full_responses,\n    }\n\n\n# %%\n# ─────────────────────────────────────────────────────────\n#  ทดสอบ Self-Consistency\n# ─────────────────────────────────────────────────────────\n\n# โจทย์ง่าย: ทดสอบว่า SC ทำงานได้\nEASY_PROBLEM = \"มีแอปเปิ้ล 24 ลูก แบ่งให้เด็ก 6 คนเท่าๆ กัน แต่ละคนได้กี่ลูก?\"\n\n# โจทย์ยาก: หลายขั้นตอน มีหน่วยซับซ้อน\nHARD_PROBLEM = \"\"\"บริษัทมีพนักงาน 3 แผนก:\n- แผนก A: 15 คน เงินเดือนเฉลี่ย 35,000 บาท\n- แผนก B: 20 คน เงินเดือนเฉลี่ย 42,000 บาท\n- แผนก C: 10 คน เงินเดือนเฉลี่ย 58,000 บาท\n\nบริษัทจะขึ้นเงินเดือนทุกคน 8%\nค่าใช้จ่ายเงินเดือนรวมต่อเดือนหลังขึ้นเงินเดือนเป็นเท่าไหร่?\"\"\"\n\n# ── ทดสอบโจทย์ง่าย ──\nprint_title(\"ทดสอบ: โจทย์ง่าย (หาร)\")\nprint_box(\"โจทย์\", EASY_PROBLEM, \"❓\")\nsc_easy_result = run_self_consistency(SC_PROMPT_V1, EASY_PROBLEM, n_samples=3)\n\n# ── ทดสอบโจทย์ยาก ──\nprint_title(\"ทดสอบ: โจทย์ยาก (คำนวณเงินเดือน)\")\nprint_box(\"โจทย์\", HARD_PROBLEM, \"❓\")\nsc_hard_result = run_self_consistency(SC_PROMPT_V1, HARD_PROBLEM, n_samples=5)\n\n# %% [markdown]\n# ---\n# ## 📘 เทคนิคที่ 2: Prompt Chaining\n#\n# ### หลักการ\n#\n# แทนที่จะเขียน Prompt ยาวๆ ครั้งเดียว → **แบ่งงานออกเป็นขั้นตอนย่อย**\n# โดย output ของแต่ละขั้น จะเป็น input ของขั้นถัดไป\n#\n# ### ทำไมถึงได้ผล?\n#\n# - AI ทำงานได้ดีขึ้นเมื่อ focus ทีละอย่าง\n# - Debug ง่าย: รู้ว่าขั้นไหนผิด\n# - แต่ละขั้นสามารถ optimize แยกกันได้\n# - ใช้ output จากขั้นก่อนเป็น context\n#\n# ### แผนผัง Lab นี้ (วิเคราะห์ Review)\n#\n# ```\n#  Review ──→ [Step 1: Extract]  ──→ JSON ข้อมูลดิบ\n#                                         │\n#                                         ▼\n#             [Step 2: Analyze] ──→ Score + Business Insight\n#                                         │\n#                                         ▼\n#             [Step 3: Report]  ──→ Executive Summary\n# ```\n#\n# ### เหมาะกับงานประเภทไหน?\n#\n# | ✅ เหมาะ | ❌ ไม่เหมาะ |\n# |---------|-----------|\n# | งานที่แบ่งขั้นได้ชัด | งานง่ายๆ 1 ขั้น |\n# | ต้องการ intermediate results | ต้องการ speed สูง |\n# | งาน document processing | ต้นทุน API ต่ำ |\n\n# %%\n# ─────────────────────────────────────────────────────────\n#  ลงทะเบียน Prompt Chain 3 ขั้นตอน ใน MLflow\n# ─────────────────────────────────────────────────────────\n\nprint_title(\"เทคนิคที่ 2: Prompt Chaining\")\n\n# ── กำหนด Template แต่ละขั้น ──\n\nCHAIN_PROMPTS = {\n\n    # ขั้นที่ 1: สกัดข้อมูลสำคัญจาก Review\n    \"chain_extract\": {\n        \"template\": \"\"\"คุณเป็นผู้เชี่ยวชาญด้านการสกัดข้อมูลจาก Review\n\nรีวิว:\n{{ review }}\n\nงาน: สกัดข้อมูลสำคัญออกมาให้ครบถ้วน ตอบเป็น JSON เท่านั้น\n\n{\n  \"positive_points\"  : [\"จุดดีที่ลูกค้ากล่าวถึง\"],\n  \"negative_points\"  : [\"จุดด้อยที่ลูกค้ากล่าวถึง\"],\n  \"overall_emotion\"  : \"positive / negative / mixed\",\n  \"has_sarcasm\"      : \"yes / no — อธิบายสั้นๆ ว่าเจอตรงไหน\",\n  \"would_recommend\"  : \"yes / no / neutral\"\n}\"\"\",\n        \"commit_message\": \"v1: สกัดข้อมูลพื้นฐานจาก Review\",\n    },\n\n    # ขั้นที่ 2: วิเคราะห์ Sentiment และ Business Impact\n    \"chain_analyze\": {\n        \"template\": \"\"\"คุณเป็นนักวิเคราะห์ Sentiment และ Business Intelligence\n\nข้อมูลที่สกัดจาก Review:\n{{ extracted }}\n\nงาน: วิเคราะห์เชิงลึก ตอบเป็น JSON เท่านั้น\n\n{\n  \"sentiment_score\"   : \"0-10 (0=แย่มาก, 10=ดีมาก)\",\n  \"urgent_fixes\"      : [\"สิ่งที่ต้องแก้เร่งด่วน เรียงตามความสำคัญ\"],\n  \"business_opportunities\" : [\"โอกาสทางธุรกิจที่เห็นได้\"],\n  \"business_risks\"    : [\"ความเสี่ยงถ้าไม่แก้ไข\"]\n}\"\"\",\n        \"commit_message\": \"v1: วิเคราะห์ Sentiment + Business Impact\",\n    },\n\n    # ขั้นที่ 3: สรุปรายงานสำหรับผู้บริหาร\n    \"chain_report\": {\n        \"template\": \"\"\"คุณเป็น Business Report Writer ผู้เชี่ยวชาญ\n\nข้อมูลดิบจาก Review:\n{{ extracted }}\n\nผลการวิเคราะห์:\n{{ analysis }}\n\nงาน: เขียน Executive Summary กระชับ ชัดเจน ใช้ตัดสินใจได้ทันที\n\nรูปแบบ:\n**HEADLINE:** [สรุปสถานการณ์ใน 1 ประโยค]\n\n**KEY FINDINGS:**\n• [ข้อค้นพบสำคัญที่ 1]\n• [ข้อค้นพบสำคัญที่ 2]\n• [ข้อค้นพบสำคัญที่ 3]\n\n**PRIORITY ACTIONS:**\n1. [สิ่งที่ต้องทำทันที]\n2. [สิ่งที่ต้องทำภายใน 1 สัปดาห์]\n3. [สิ่งที่ต้องทำภายใน 1 เดือน]\n\n**RISK LEVEL:** LOW / MEDIUM / HIGH\n**REASON:** [เหตุผล 1-2 ประโยค]\"\"\",\n        \"commit_message\": \"v1: สร้าง Executive Summary Report\",\n    },\n}\n\n# ── ลงทะเบียนทุก Prompt ใน MLflow ──\nregistered_chain_prompts = {}\n\nfor prompt_name, prompt_info in CHAIN_PROMPTS.items():\n    registered = mlflow.genai.register_prompt(\n        name=prompt_name,\n        template=prompt_info[\"template\"],\n        commit_message=prompt_info[\"commit_message\"],\n    )\n    registered_chain_prompts[prompt_name] = registered\n    track_version(prompt_name, registered.version)\n\n    print(f\"  ✅ ลงทะเบียน: '{prompt_name}' version {registered.version}\")\n\nprint_divider()\nprint(f\"  📦 ลงทะเบียน Prompt Chain ทั้งหมด {len(CHAIN_PROMPTS)} ขั้นตอน\")\n\n# %%\n# ─────────────────────────────────────────────────────────\n#  Chain Runner: รัน 3 ขั้นตอนต่อเนื่องกัน\n# ─────────────────────────────────────────────────────────\n\ndef run_prompt_chain(review_text: str) -> Dict:\n    \"\"\"\n    รัน Prompt Chain 3 ขั้น: Extract → Analyze → Report\n\n    Output ของแต่ละขั้นจะถูกส่งต่อเป็น Input ของขั้นถัดไป\n\n    Parameters:\n        review_text : รีวิวที่ต้องการวิเคราะห์\n\n    Returns:\n        dict: ผลลัพธ์จากทุกขั้น {\n            \"chain_extract\": \"...\",\n            \"chain_analyze\": \"...\",\n            \"chain_report\" : \"...\",\n        }\n    \"\"\"\n    step_results = {}\n\n    step_configs = [\n        {\n            \"name\"  : \"chain_extract\",\n            \"emoji\" : \"🔍\",\n            \"label\" : \"สกัดข้อมูล (Extract)\",\n            \"build_prompt\": lambda p, _: p.format(review=review_text),\n        },\n        {\n            \"name\"  : \"chain_analyze\",\n            \"emoji\" : \"📊\",\n            \"label\" : \"วิเคราะห์ Sentiment (Analyze)\",\n            \"build_prompt\": lambda p, r: p.format(extracted=r.get(\"chain_extract\", \"\")),\n        },\n        {\n            \"name\"  : \"chain_report\",\n            \"emoji\" : \"📋\",\n            \"label\" : \"สรุปรายงาน (Report)\",\n            \"build_prompt\": lambda p, r: p.format(\n                extracted = r.get(\"chain_extract\", \"\"),\n                analysis  = r.get(\"chain_analyze\", \"\"),\n            ),\n        },\n    ]\n\n    print_title(f\"รัน Prompt Chain ({len(step_configs)} ขั้นตอน)\", 2)\n\n    for step_num, step in enumerate(step_configs, start=1):\n        # โหลด prompt เวอร์ชันล่าสุด\n        prompt = load_latest_prompt(step[\"name\"])\n\n        # สร้าง prompt จริงโดยแทนค่า\n        filled = step[\"build_prompt\"](prompt, step_results)\n\n        # เรียก AI และจับเวลา\n        start_time = time.time()\n        result     = call_gemini(filled)\n        elapsed    = time.time() - start_time\n\n        step_results[step[\"name\"]] = result\n\n        # แสดงผลแต่ละขั้น\n        print_box(\n            title   = f\"ขั้นที่ {step_num}: {step['label']}  ⏱️ {elapsed:.1f}s\",\n            content = result,\n            emoji   = step[\"emoji\"],\n        )\n\n        time.sleep(1.0)\n\n    print(f\"\\n  ✅ Chain สำเร็จ! ผ่านทั้ง {len(step_configs)} ขั้นตอน\")\n    return step_results\n\n\n# %%\n# ─────────────────────────────────────────────────────────\n#  ทดสอบ Prompt Chain ด้วย Review แบบ Sarcastic (ประชดประชัน)\n# ─────────────────────────────────────────────────────────\n\n# Review ที่มีการประชดประชัน ทดสอบว่า Chain สามารถจับได้ไหม\nSARCASTIC_REVIEW = \"\"\"\nว้าว! ประทับใจมากจริงๆ ครับ สั่งไปเมื่อ 3 อาทิตย์ที่แล้ว เพิ่งได้รับวันนี้เอง\nช้าแค่นี้เองนะ ระดับ World Class มากๆ\n\nตัวสินค้าก็ดีครับ ถ้าไม่นับว่ามันแตกมาครึ่งหนึ่ง ก็ถือว่าโอเคนะ\nจ่ายไป 2,500 บาท ได้ของที่ใช้ได้แค่ 3 วัน คุ้มมากๆ\n\nCustomer Service ตอบเร็วมากภายใน 5 วันทำการ ประทับใจในความรวดเร็ว\n\nโดยรวมถ้าชอบความตื่นเต้นว่าของจะมาถึงมือหรือเปล่า แนะนำเลยครับ\n\"\"\"\n\nprint_box(\n    title   = \"Review ทดสอบ (แบบประชดประชัน)\",\n    content = SARCASTIC_REVIEW,\n    emoji   = \"💬\",\n)\n\nchain_results = run_prompt_chain(SARCASTIC_REVIEW)\n\n# %% [markdown]\n# ---\n# ## 📘 เทคนิคที่ 3: Self-Critique\n#\n# ### หลักการ\n#\n# ให้ AI **ตรวจและวิพากษ์คำตอบตัวเอง** แล้วนำผลวิพากษ์มาปรับปรุง\n#\n# เปรียบเหมือนนักเขียนที่เขียนร่างแรก → แก้ไขเอง → ร่างสุดท้ายดีขึ้น\n#\n# ### แผนผัง\n#\n# ```\n#  คำถาม ──→ [Generate]  ──→  คำตอบ Draft\n#                                    │\n#                                    ▼\n#             [Critique]  ──→  จุดอ่อน / ข้อผิดพลาด\n#                                    │\n#                                    ▼\n#             [Revise]    ──→  คำตอบ Final (ดีขึ้น)\n#                                    │\n#                            (ทำซ้ำได้ N รอบ)\n# ```\n#\n# ### ข้อดี vs ข้อเสีย\n#\n# | ✅ ข้อดี | ⚠️ ข้อเสีย |\n# |---------|----------|\n# | ลด bias และ logical fallacy | ใช้ token มากกว่า 3x |\n# | ได้คำตอบรอบด้านขึ้น | ช้ากว่า |\n# | เหมาะกับหัวข้อ controversial | อาจ over-critique |\n\n# %%\n# ─────────────────────────────────────────────────────────\n#  ลงทะเบียน Self-Critique Prompts ใน MLflow\n# ─────────────────────────────────────────────────────────\n\nprint_title(\"เทคนิคที่ 3: Self-Critique\")\n\nCRITIQUE_PROMPTS = {\n\n    # ขั้นที่ 1: สร้างคำตอบเบื้องต้น\n    \"crit_generate\": {\n        \"template\": \"\"\"คุณเป็นผู้เชี่ยวชาญที่ให้คำแนะนำอย่างรอบคอบและสมดุล\n\nคำถาม: {{ task }}\n\nตอบคำถามนี้อย่างครอบคลุม พิจารณาหลายมุมมอง\nพร้อมให้เหตุผลชัดเจนสำหรับทุกข้อ\"\"\",\n        \"commit_message\": \"v1: Generate initial answer\",\n    },\n\n    # ขั้นที่ 2: วิพากษ์คำตอบอย่างเข้มข้น\n    \"crit_critic\": {\n        \"template\": \"\"\"คุณเป็น Devil's Advocate ผู้เชี่ยวชาญ — หน้าที่คือหาจุดอ่อนทุกอย่าง\n\nโจทย์เดิม:\n{{ task }}\n\nคำตอบที่ต้องวิพากษ์:\n{{ answer }}\n\nตรวจสอบอย่างเข้มข้นตามหัวข้อเหล่านี้:\n\n□ Logical Fallacies  — มีการใช้ตรรกะที่ผิดพลาดไหม?\n□ Unsupported Claims — มีการอ้างสิ่งที่ไม่มีหลักฐานสนับสนุนไหม?\n□ Missing Perspectives — ขาดมุมมองสำคัญอะไรไปบ้าง?\n□ Bias — มีความลำเอียงไหม? (เช่น เข้าข้างฝ่ายใดฝ่ายหนึ่ง)\n□ Feasibility — สิ่งที่แนะนำทำได้จริงในทางปฏิบัติไหม?\n\n⚠️ ข้อกำหนด: ต้องหาปัญหาอย่างน้อย 3 ข้อ ห้ามพูดว่าดีโดยไม่ชี้ข้อผิด\n\nตอบในรูปแบบ:\nISSUES:\n- [ปัญหาที่ 1: อธิบายชัดเจน]\n- [ปัญหาที่ 2: อธิบายชัดเจน]\n- [ปัญหาที่ 3: อธิบายชัดเจน]\n\nVERDICT: WEAK / ACCEPTABLE / STRONG\nSUMMARY: [สรุป 2 ประโยค ว่าทำไมถึงให้ verdict นี้]\"\"\",\n        \"commit_message\": \"v1: Adversarial critic with structured output\",\n    },\n\n    # ขั้นที่ 3: ปรับปรุงคำตอบโดยใช้ผลวิพากษ์\n    \"crit_revise\": {\n        \"template\": \"\"\"คุณเป็นผู้เชี่ยวชาญที่รับผลวิพากษ์มาปรับปรุงคำตอบ\n\nโจทย์เดิม:\n{{ task }}\n\nคำตอบเดิม:\n{{ answer }}\n\nผลวิพากษ์จาก Critic:\n{{ critique }}\n\nงาน: ปรับปรุงคำตอบโดย:\n1. แก้ไขทุกปัญหาที่ถูกชี้ใน ISSUES\n2. เพิ่มมุมมองที่ขาดหายไป\n3. รักษาจุดแข็งของคำตอบเดิม\n\nตอบในรูปแบบ:\nWHAT_I_FIXED:\n- [สิ่งที่แก้ไข 1]\n- [สิ่งที่แก้ไข 2]\n\nREVISED_ANSWER:\n[คำตอบใหม่ที่ปรับปรุงแล้ว]\n\nCONFIDENCE: LOW / MEDIUM / HIGH\nREASON: [เหตุผล 1 ประโยค]\"\"\",\n        \"commit_message\": \"v1: Structured revision based on critique\",\n    },\n}\n\n# ── ลงทะเบียนทุก Prompt ──\nfor prompt_name, prompt_info in CRITIQUE_PROMPTS.items():\n    registered = mlflow.genai.register_prompt(\n        name=prompt_name,\n        template=prompt_info[\"template\"],\n        commit_message=prompt_info[\"commit_message\"],\n    )\n    track_version(prompt_name, registered.version)\n    print(f\"  ✅ ลงทะเบียน: '{prompt_name}' version {registered.version}\")\n\n# %%\n# ─────────────────────────────────────────────────────────\n#  Self-Critique Runner\n# ─────────────────────────────────────────────────────────\n\ndef run_self_critique(task: str, n_rounds: int = 1) -> Dict:\n    \"\"\"\n    รัน Self-Critique Loop: Generate → Critique → Revise\n\n    Parameters:\n        task     : คำถาม / โจทย์ที่ต้องการคำตอบ\n        n_rounds : จำนวนรอบที่วิพากษ์และแก้ไข (มากขึ้น = ดีขึ้น แต่ช้าลง)\n\n    Returns:\n        dict: {\n            \"initial_answer\" : คำตอบแรกก่อน critique\n            \"final_answer\"   : คำตอบสุดท้ายหลัง revise ทุกรอบ\n            \"rounds\"         : ผลของแต่ละรอบ\n        }\n    \"\"\"\n    # โหลด prompts ล่าสุด\n    generate_prompt = load_latest_prompt(\"crit_generate\")\n    critic_prompt   = load_latest_prompt(\"crit_critic\")\n    revise_prompt   = load_latest_prompt(\"crit_revise\")\n\n    print_title(f\"Self-Critique ({n_rounds} รอบ)\", 2)\n\n    # ── ขั้นที่ 1: สร้างคำตอบแรก ──\n    print(\"    📝 กำลังสร้างคำตอบเบื้องต้น...\")\n    initial_answer = call_gemini(\n        generate_prompt.format(task=task),\n        temperature=0.2,\n    )\n    print_box(\"ขั้นที่ 1: คำตอบเบื้องต้น (Draft)\", initial_answer, \"📝\")\n    time.sleep(1.0)\n\n    current_answer = initial_answer\n    all_rounds_data = []\n\n    # ── รัน N รอบ Critique + Revise ──\n    for round_num in range(1, n_rounds + 1):\n        print_title(f\"รอบที่ {round_num}/{n_rounds}\", 2)\n\n        # Critique: วิพากษ์คำตอบ\n        print(\"    ⚔️  กำลังวิพากษ์คำตอบ...\")\n        critique = call_gemini(\n            critic_prompt.format(task=task, answer=current_answer),\n            temperature=0.1,  # temperature ต่ำ = วิพากษ์อย่างสม่ำเสมอ\n        )\n        print_box(f\"ขั้นที่ 2: ผลวิพากษ์ (รอบ {round_num})\", critique, \"⚔️\")\n        time.sleep(1.0)\n\n        # Revise: ปรับปรุงคำตอบ\n        print(\"    ✏️  กำลังปรับปรุงคำตอบ...\")\n        revised = call_gemini(\n            revise_prompt.format(task=task, answer=current_answer, critique=critique),\n            temperature=0.2,\n        )\n        print_box(f\"ขั้นที่ 3: คำตอบที่ปรับปรุงแล้ว (รอบ {round_num})\", revised, \"✨\")\n        time.sleep(1.0)\n\n        # บันทึกผลรอบนี้\n        verdict = next(\n            (v for v in [\"STRONG\", \"ACCEPTABLE\", \"WEAK\"] if v in critique),\n            \"UNKNOWN\",\n        )\n        all_rounds_data.append({\n            \"round\"  : round_num,\n            \"critique\": critique,\n            \"revised\" : revised,\n            \"verdict\" : verdict,\n        })\n        current_answer = revised  # ใช้คำตอบใหม่ในรอบถัดไป\n\n    # สรุปผล\n    print_title(\"สรุป Self-Critique\", 2)\n    for rd in all_rounds_data:\n        verdict_emoji = {\"STRONG\": \"💪\", \"ACCEPTABLE\": \"👍\", \"WEAK\": \"⚠️\"}.get(rd[\"verdict\"], \"❓\")\n        print_metric(f\"รอบ {rd['round']} Verdict\", rd[\"verdict\"], \"\", verdict_emoji)\n\n    return {\n        \"initial_answer\": initial_answer,\n        \"final_answer\"  : current_answer,\n        \"rounds\"        : all_rounds_data,\n        \"task\"          : task,\n    }\n\n\n# %%\n# ─────────────────────────────────────────────────────────\n#  ทดสอบ Self-Critique ด้วยหัวข้อที่ต้องพิจารณาหลายมุม\n# ─────────────────────────────────────────────────────────\n\nCOMPLEX_TASK = \"\"\"\nบริษัท Startup ด้าน FinTech (45 คน, กระจาย 3 จังหวัด) ควรใช้นโยบาย\nRemote Work 100% หรือ มาออฟฟิศทุกวัน?\n\nโปรดพิจารณาจาก 3 มุมมอง: นายจ้าง, ลูกจ้าง, และลูกค้า\n\"\"\"\n\nprint_box(\"โจทย์ทดสอบ Self-Critique\", COMPLEX_TASK, \"❓\")\n\ncritique_result = run_self_critique(COMPLEX_TASK, n_rounds=2)\n\n# %% [markdown]\n# ---\n# ## 🔁 Reflection Loop: ปรับปรุง Prompt อัตโนมัติ\n#\n# ### หลักการ\n#\n# **Reflection** คือการให้ AI วิเคราะห์ว่าทำไม Prompt ถึงให้ผลลัพธ์ไม่ดี\n# แล้วสร้าง Prompt ใหม่ที่แก้ปัญหานั้น → วนซ้ำจนได้ accuracy ตามเป้า\n#\n# ### แผนผัง\n#\n# ```\n#  Prompt vN ──→ [ทดสอบกับ Dataset] ──→ Accuracy ≥ Target?\n#                                              │\n#                                        YES   │   NO\n#                                         ↓    │    ↓\n#                                        DONE  │  [Reflect]\n#                                              │    ↓\n#                                              │  วิเคราะห์ข้อที่ผิด\n#                                              │    ↓\n#                                              │  สร้าง Prompt vN+1\n#                                              │    ↓\n#                                              └──→ Loop\n# ```\n#\n# ### Dataset ที่ใช้ทดสอบ\n#\n# | ระดับ | จำนวน | ตัวอย่าง |\n# |-------|-------|---------|\n# | easy  | 2 ข้อ | หาร, คูณ ธรรมดา |\n# | medium| 2 ข้อ | หลายขั้นตอน, ส่วนลด |\n# | hard  | 2 ข้อ | ทำงานร่วม, ที่นั่ง |\n\n# %%\n# ─────────────────────────────────────────────────────────\n#  Test Dataset: 6 โจทย์ 3 ระดับ\n# ─────────────────────────────────────────────────────────\n\nprint_title(\"Reflection Loop: ปรับปรุง Prompt อัตโนมัติ\")\n\nTEST_DATASET = [\n    # ── ระดับ Easy ──\n    {\n        \"problem\"   : \"มีส้ม 24 ลูก แบ่งให้เด็ก 6 คนเท่าๆ กัน แต่ละคนได้กี่ลูก?\",\n        \"expected\"  : \"4\",\n        \"difficulty\": \"easy\",\n    },\n    {\n        \"problem\"   : \"รถยนต์วิ่งด้วยความเร็ว 80 กม./ชม. นาน 3 ชั่วโมง วิ่งได้กี่กิโลเมตร?\",\n        \"expected\"  : \"240\",\n        \"difficulty\": \"easy\",\n    },\n\n    # ── ระดับ Medium ──\n    {\n        \"problem\"   : \"เสื้อ ราคา 350 บาท ซื้อ 3 ตัว และกางเกง ราคา 450 บาท ซื้อ 2 ตัว \"\n                      \"ถ้าซื้อเกิน 1,500 บาท ได้ส่วนลด 15% จ่ายทั้งหมดเท่าไหร่?\",\n        \"expected\"  : \"1785\",\n        \"difficulty\": \"medium\",\n    },\n    {\n        \"problem\"   : \"รถ 5 คัน วิ่งรวมกัน 450 กม./วัน อัตราสิ้นเปลือง 12 กม./ลิตร \"\n                      \"น้ำมันราคา 42 บาท/ลิตร ค่าน้ำมันรวมทั้งหมดต่อวันเท่าไหร่?\",\n        \"expected\"  : \"1575\",\n        \"difficulty\": \"medium\",\n    },\n\n    # ── ระดับ Hard ──\n    {\n        \"problem\"   : \"ทีม A ทำงานได้ 1/30 ของงานต่อวัน ทีม B ทำได้ 1/45 ของงานต่อวัน \"\n                      \"ถ้าทำงานพร้อมกัน ใช้กี่วันถึงเสร็จ? (ปัดขึ้นเป็นจำนวนเต็ม)\",\n        \"expected\"  : \"18\",\n        \"difficulty\": \"hard\",\n    },\n    {\n        \"problem\"   : \"ร้านอาหารมี: โต๊ะเดี่ยว 8 โต๊ะ (2 คน/โต๊ะ), โต๊ะคู่ 5 โต๊ะ (4 คน/โต๊ะ), \"\n                      \"โต๊ะใหญ่ 3 โต๊ะ (8 คน/โต๊ะ). ตอนนี้มีลูกค้า 35 คน และจะมาเพิ่มอีก \"\n                      \"2 กลุ่ม กลุ่มละ 7 คน เหลือที่นั่งว่างกี่ที่?\",\n        \"expected\"  : \"5\",\n        \"difficulty\": \"hard\",\n    },\n]\n\n# แสดง Dataset ในตาราง\nprint_table(\n    headers = [\"#\", \"Difficulty\", \"Expected\", \"โจทย์ (ย่อ)\"],\n    rows    = [\n        [\n            str(i + 1),\n            item[\"difficulty\"],\n            item[\"expected\"],\n            item[\"problem\"][:45] + \"...\",\n        ]\n        for i, item in enumerate(TEST_DATASET)\n    ],\n    title = \"Test Dataset (6 โจทย์)\",\n)\n\n# %%\n# ─────────────────────────────────────────────────────────\n#  Evaluate Function: ทดสอบ Prompt กับ Dataset\n# ─────────────────────────────────────────────────────────\n\ndef evaluate_prompt(template: str,\n                    dataset: List[Dict],\n                    n_samples: int = 3,\n                    label: str = \"Prompt\") -> Dict:\n    \"\"\"\n    ทดสอบ SC Prompt กับ dataset ทั้งหมด และคืนค่า accuracy\n\n    Parameters:\n        template  : Prompt template ที่จะทดสอบ\n        dataset   : list ของ {problem, expected, difficulty}\n        n_samples : จำนวน SC paths ต่อโจทย์\n        label     : ชื่อสำหรับแสดงผล\n\n    Returns:\n        dict: {\n            \"accuracy\"  : overall accuracy (%)\n            \"results\"   : ผลแต่ละข้อ\n            \"breakdown\" : accuracy แยกตาม difficulty\n        }\n    \"\"\"\n    correct_count = 0\n    all_results   = []\n\n    print_title(f\"ประเมิน: {label}\", 2)\n\n    for idx, item in enumerate(dataset, start=1):\n        # รัน SC (verbose=False เพื่อแสดงผลสั้น)\n        sc_result = run_self_consistency(\n            template    = template,\n            problem     = item[\"problem\"],\n            n_samples   = n_samples,\n            temperature = 0.7,\n            show_details = False,\n        )\n\n        # เปรียบเทียบคำตอบ\n        predicted = sc_result[\"final_answer\"].replace(\",\", \"\").strip()\n        expected  = item[\"expected\"].strip()\n        is_correct = (predicted == expected)\n\n        if is_correct:\n            correct_count += 1\n\n        all_results.append({\n            \"idx\"        : idx,\n            \"problem\"    : item[\"problem\"][:45],\n            \"expected\"   : expected,\n            \"predicted\"  : predicted,\n            \"confidence\" : sc_result[\"confidence\"],\n            \"difficulty\" : item[\"difficulty\"],\n            \"correct\"    : is_correct,\n        })\n\n        # แสดงผลแต่ละข้อ\n        status = \"✅\" if is_correct else \"❌\"\n        print(\n            f\"      {status} Q{idx} [{item['difficulty']:6s}]  \"\n            f\"Expected={expected:>6s}  Got={predicted:>6s}  \"\n            f\"Confidence={sc_result['confidence']:.0%}\"\n        )\n\n    # ── คำนวณ Accuracy รวม ──\n    overall_accuracy = (correct_count / len(dataset)) * 100\n\n    # ── คำนวณ Accuracy แยกตาม Difficulty ──\n    breakdown = {}\n    for difficulty in [\"easy\", \"medium\", \"hard\"]:\n        diff_results = [r for r in all_results if r[\"difficulty\"] == difficulty]\n        diff_correct = sum(1 for r in diff_results if r[\"correct\"])\n        diff_total   = len(diff_results)\n        diff_acc     = (diff_correct / diff_total * 100) if diff_total > 0 else 0\n        breakdown[difficulty] = {\n            \"correct\" : diff_correct,\n            \"total\"   : diff_total,\n            \"accuracy\": diff_acc,\n        }\n\n    # ── แสดงผลสรุป ──\n    print_table(\n        headers = [\"#\", \"Difficulty\", \"Expected\", \"Got\", \"Confidence\", \"Result\"],\n        rows    = [\n            [\n                str(r[\"idx\"]),\n                r[\"difficulty\"],\n                r[\"expected\"],\n                r[\"predicted\"],\n                f\"{r['confidence']:.0%}\",\n                \"✅\" if r[\"correct\"] else \"❌\",\n            ]\n            for r in all_results\n        ],\n        title = f\"ผลการประเมิน — {label}\",\n    )\n\n    # Summary box\n    difficulty_emoji = {\"easy\": \"🟢\", \"medium\": \"🟡\", \"hard\": \"🔴\"}\n    summary_lines = [f\"Overall Accuracy: {overall_accuracy:.1f}%  ({correct_count}/{len(dataset)})\\n\"]\n    for diff, info in breakdown.items():\n        filled = int(info[\"accuracy\"] / 10)\n        bar    = \"█\" * filled + \"░\" * (10 - filled)\n        summary_lines.append(\n            f\"{difficulty_emoji[diff]} {diff.capitalize():8s}:  {bar}  \"\n            f\"{info['accuracy']:.0f}%  ({info['correct']}/{info['total']})\"\n        )\n\n    print_box(\n        title   = f\"สรุป Accuracy — {label}\",\n        content = \"\\n\".join(summary_lines),\n        emoji   = \"🎯\",\n    )\n\n    return {\n        \"accuracy\" : overall_accuracy,\n        \"results\"  : all_results,\n        \"breakdown\": breakdown,\n    }\n\n\n# %%\n# ─────────────────────────────────────────────────────────\n#  Reflect Function: วิเคราะห์ข้อผิดพลาด → สร้าง Prompt ใหม่\n# ─────────────────────────────────────────────────────────\n\ndef reflect_and_improve_prompt(current_template: str,\n                               eval_result: Dict) -> str:\n    \"\"\"\n    วิเคราะห์ว่า Prompt ผิดพลาดตรงไหน และสร้าง Prompt ที่ดีขึ้น\n\n    Parameters:\n        current_template : Prompt เวอร์ชันปัจจุบัน\n        eval_result      : ผลการ evaluate (จาก evaluate_prompt)\n\n    Returns:\n        str: Prompt template ใหม่ที่ควรดีขึ้น\n             (ถ้าสกัดไม่ได้ จะคืน template เดิม)\n    \"\"\"\n    wrong_items = [r for r in eval_result[\"results\"] if not r[\"correct\"]]\n\n    if not wrong_items:\n        print(\"  🎉 Accuracy 100% — ไม่จำเป็นต้องปรับปรุง!\")\n        return current_template\n\n    # สร้าง Reflection Prompt\n    wrong_summary = json.dumps(wrong_items[:4], ensure_ascii=False, indent=2)\n\n    reflection_prompt = f\"\"\"คุณเป็น Prompt Engineer ผู้เชี่ยวชาญด้าน Math Reasoning\n\n## Prompt ปัจจุบัน:\n```\n{current_template}\n```\n\n## ผลการทดสอบ:\n- Overall Accuracy: {eval_result['accuracy']:.1f}%\n- ข้อที่ผิด {len(wrong_items)} ข้อ\n\n## รายละเอียดข้อที่ผิด:\n{wrong_summary}\n\n## งานของคุณ:\n1. วิเคราะห์ pattern ว่าทำไม Prompt ปัจจุบันถึงให้คำตอบผิด\n2. ระบุจุดอ่อน 2-3 จุดของ Prompt ปัจจุบัน\n3. สร้าง Prompt ใหม่ที่แก้ไขจุดอ่อนเหล่านั้น\n\n## กฎที่ต้องรักษา:\n- ต้องมี {{{{ problem }}}} ใน prompt (สำหรับแทน placeholder)\n- ให้ AI แสดง reasoning ก่อนสรุปคำตอบ\n- กำหนด output format ชัดเจน\n- คำตอบสุดท้ายต้องเป็นตัวเลข\n\n## รูปแบบ output:\nขึ้นต้น Prompt ใหม่ด้วย [NEW_PROMPT_START]\nจบด้วย [NEW_PROMPT_END]\"\"\"\n\n    print_title(\"กำลัง Reflect...\", 3)\n    reflection_response = call_gemini(reflection_prompt, temperature=0.3, max_tokens=4096)\n\n    print_box(\"ผล Reflection Analysis\", reflection_response, \"🔍\")\n\n    # สกัด Prompt ใหม่\n    if \"[NEW_PROMPT_START]\" in reflection_response and \"[NEW_PROMPT_END]\" in reflection_response:\n        start_idx  = reflection_response.index(\"[NEW_PROMPT_START]\") + len(\"[NEW_PROMPT_START]\")\n        end_idx    = reflection_response.index(\"[NEW_PROMPT_END]\")\n        new_prompt = reflection_response[start_idx:end_idx].strip()\n\n        print_box(\n            title   = f\"Prompt ใหม่ที่สร้างขึ้น ({len(new_prompt)} ตัวอักษร)\",\n            content = new_prompt,\n            emoji   = \"🆕\",\n        )\n        return new_prompt\n\n    print(\"    ⚠️  ไม่พบ Prompt ใหม่ใน response — ใช้ template เดิม\")\n    return current_template\n\n\n# %%\n# ─────────────────────────────────────────────────────────\n#  Optimization Loop: วนทดสอบและปรับปรุงจนถึง Target\n# ─────────────────────────────────────────────────────────\n\ndef run_optimization_loop(prompt_name: str,\n                           initial_template: str,\n                           dataset: List[Dict],\n                           target_accuracy: float = 83.0,\n                           max_iterations: int = 3,\n                           n_samples: int = 3) -> Dict:\n    \"\"\"\n    วน Loop: Evaluate → Reflect → Improve → Register → Repeat\n\n    Parameters:\n        prompt_name      : ชื่อ Prompt ใน MLflow Registry\n        initial_template : Prompt เริ่มต้น\n        dataset          : Dataset สำหรับทดสอบ\n        target_accuracy  : เป้าหมาย accuracy (%)\n        max_iterations   : รอบสูงสุด (ป้องกัน loop ไม่สิ้นสุด)\n        n_samples        : SC paths ต่อโจทย์\n\n    Returns:\n        dict: สรุปผลทุก iteration\n    \"\"\"\n    optimization_history = []\n    current_template     = initial_template\n\n    print_title(f\"Optimization Loop — เป้าหมาย {target_accuracy}%\", 2)\n    print_divider(\"═\")\n\n    for iteration in range(1, max_iterations + 1):\n        print(f\"\\n  🔄 Iteration {iteration}/{max_iterations}\")\n        print_divider()\n\n        # ── ทดสอบ Prompt ปัจจุบัน ──\n        eval_result = evaluate_prompt(\n            template  = current_template,\n            dataset   = dataset,\n            n_samples = n_samples,\n            label     = f\"Iteration {iteration}\",\n        )\n\n        # บันทึกประวัติ\n        optimization_history.append({\n            \"iteration\": iteration,\n            \"accuracy\" : eval_result[\"accuracy\"],\n            \"breakdown\": eval_result[\"breakdown\"],\n        })\n\n        # ── ตรวจสอบว่าถึง Target แล้วหรือยัง ──\n        if eval_result[\"accuracy\"] >= target_accuracy:\n            print(f\"\\n  🎉 ถึงเป้าหมายแล้ว! {eval_result['accuracy']:.1f}% ≥ {target_accuracy}%\")\n            break\n\n        print(f\"\\n  🔍 ยังไม่ถึงเป้า ({eval_result['accuracy']:.1f}% < {target_accuracy}%)\")\n        print(    \"     กำลัง Reflect และสร้าง Prompt ใหม่...\")\n\n        # ── Reflect และสร้าง Prompt ใหม่ ──\n        new_template = reflect_and_improve_prompt(current_template, eval_result)\n\n        if new_template != current_template:\n            # ลงทะเบียน Prompt ใหม่ใน MLflow\n            new_registered = mlflow.genai.register_prompt(\n                name=prompt_name,\n                template=new_template,\n                commit_message=(\n                    f\"Reflection iter {iteration}: \"\n                    f\"accuracy {eval_result['accuracy']:.1f}% → improving\"\n                ),\n            )\n            track_version(prompt_name, new_registered.version)\n            print(f\"  📦 ลงทะเบียน version ใหม่: v{new_registered.version}\")\n            current_template = new_template\n        else:\n            print(\"  ⚠️  Prompt ไม่เปลี่ยนแปลง — หยุด Loop\")\n            break\n\n        time.sleep(2.0)\n\n    # ── แสดง History ──\n    print_title(\"ประวัติการ Optimize\", 2)\n    print_table(\n        headers = [\"Iter\", \"Accuracy\", \"Easy\", \"Medium\", \"Hard\"],\n        rows    = [\n            [\n                str(h[\"iteration\"]),\n                f\"{h['accuracy']:.1f}%\",\n                f\"{h['breakdown']['easy']['accuracy']:.0f}%\",\n                f\"{h['breakdown']['medium']['accuracy']:.0f}%\",\n                f\"{h['breakdown']['hard']['accuracy']:.0f}%\",\n            ]\n            for h in optimization_history\n        ],\n        title = \"Accuracy แต่ละ Iteration\",\n    )\n\n    # แสดงการปรับปรุงรวม\n    if len(optimization_history) > 1:\n        total_improvement = (\n            optimization_history[-1][\"accuracy\"] - optimization_history[0][\"accuracy\"]\n        )\n        emoji = \"📈\" if total_improvement >= 0 else \"📉\"\n        print_metric(\"การปรับปรุงรวม\", total_improvement, \"%\", emoji)\n\n    return {\n        \"history\"          : optimization_history,\n        \"final_template\"   : current_template,\n        \"final_accuracy\"   : optimization_history[-1][\"accuracy\"],\n        \"total_iterations\" : len(optimization_history),\n    }\n\n\n# %%\n# ─────────────────────────────────────────────────────────\n#  รัน Optimization Loop จริง!\n# ─────────────────────────────────────────────────────────\n\nprint_title(\"เริ่ม Optimization Loop!\")\n\nloop_result = run_optimization_loop(\n    prompt_name      = SC_PROMPT_NAME,\n    initial_template = SC_PROMPT_V1,\n    dataset          = TEST_DATASET,\n    target_accuracy  = 83.0,    # เป้าหมาย: ตอบถูก 5/6 ข้อขึ้นไป\n    max_iterations   = 3,       # ลองปรับปรุงสูงสุด 3 รอบ\n    n_samples        = 3,       # SC 3 paths ต่อโจทย์ (ประหยัด cost)\n)\n\n# %% [markdown]\n# ---\n# ## 📊 ส่วนสุดท้าย: บันทึกผลใน MLflow และสรุปรวม\n#\n# MLflow ช่วยให้เราติดตามและเปรียบเทียบผลการทดลองทั้งหมดได้\n# รวมถึง Prompt versions, metrics, และ parameters ต่างๆ\n\n# %%\n# ─────────────────────────────────────────────────────────\n#  บันทึกผลทุกอย่างใน MLflow\n# ─────────────────────────────────────────────────────────\n\nprint_title(\"บันทึกผลใน MLflow\")\n\nwith mlflow.start_run(run_name=\"prompt_engineering_lab_final\"):\n\n    # ── SC Optimization Metrics ──\n    for hist in loop_result[\"history\"]:\n        mlflow.log_metric(\n            key  = f\"sc_accuracy_iter{hist['iteration']}\",\n            value= hist[\"accuracy\"],\n        )\n\n    mlflow.log_metric(\"sc_final_accuracy\",  loop_result[\"final_accuracy\"])\n    mlflow.log_metric(\"sc_total_iterations\", loop_result[\"total_iterations\"])\n\n    # ── Prompt Chaining Metrics ──\n    mlflow.log_metric(\"chain_n_steps\",       3)\n    mlflow.log_metric(\"chain_report_length\", len(chain_results.get(\"chain_report\", \"\")))\n\n    # ── Self-Critique Metrics ──\n    mlflow.log_metric(\"critique_n_rounds\",    len(critique_result[\"rounds\"]))\n    mlflow.log_metric(\"critique_final_length\", len(critique_result[\"final_answer\"]))\n\n    # ── Parameters ──\n    mlflow.log_param(\"model_id\",     MODEL_ID)\n    mlflow.log_param(\"techniques\",   \"Self-Consistency, Prompt-Chaining, Self-Critique\")\n    mlflow.log_param(\"test_size\",    len(TEST_DATASET))\n    mlflow.log_param(\"sc_n_samples\", 3)\n    mlflow.log_param(\"target_acc\",   83.0)\n\n    print(\"  ✅ บันทึก Metrics สำเร็จ:\")\n    print(f\"     - SC final accuracy: {loop_result['final_accuracy']:.1f}%\")\n    print(f\"     - SC iterations    : {loop_result['total_iterations']}\")\n    print(f\"     - Chain steps      : 3\")\n    print(f\"     - Critique rounds  : {len(critique_result['rounds'])}\")\n\n\n# %%\n# ─────────────────────────────────────────────────────────\n#  🏆 Dashboard สรุปผลทั้งหมด\n# ─────────────────────────────────────────────────────────\n\nprint_title(\"🏆 Lab Dashboard — สรุปผลทั้งหมด\")\n\n# ── ตาราง Technique Results ──\nsarcasm_detected = (\n    \"sarcasm\" in chain_results.get(\"chain_extract\", \"\").lower()\n    or \"ประชด\" in chain_results.get(\"chain_extract\", \"\")\n    or \"yes\" in chain_results.get(\"chain_extract\", \"\").lower()\n)\n\nprint_table(\n    headers = [\"เทคนิค\", \"สิ่งที่วัด\", \"ผลลัพธ์\"],\n    rows    = [\n        [\n            \"Self-Consistency\",\n            \"Final Accuracy\",\n            f\"{loop_result['final_accuracy']:.1f}%\",\n        ],\n        [\n            \"Prompt Chaining\",\n            \"Sarcasm Detected\",\n            \"✅ ใช่\" if sarcasm_detected else \"❓ ไม่แน่ใจ\",\n        ],\n        [\n            \"Self-Critique\",\n            \"จำนวนรอบ Revise\",\n            f\"{len(critique_result['rounds'])} รอบ\",\n        ],\n    ],\n    title = \"ผลลัพธ์แต่ละเทคนิค\",\n)\n\n# ── SC Optimization Progress ──\nprint_title(\"SC Optimization Progress\", 2)\nfor hist in loop_result[\"history\"]:\n    print_progress(\n        current = int(hist[\"accuracy\"]),\n        total   = 100,\n        label   = f\"Iter {hist['iteration']}: {hist['accuracy']:.1f}%\",\n    )\n\n# ── MLflow Registry Summary ──\nprint_title(\"MLflow Prompt Registry\", 2)\n\nall_prompt_names = [\n    SC_PROMPT_NAME,\n    \"chain_extract\",\n    \"chain_analyze\",\n    \"chain_report\",\n    \"crit_generate\",\n    \"crit_critic\",\n    \"crit_revise\",\n]\n\nprint_table(\n    headers = [\"Prompt Name\", \"Latest Version\", \"เทคนิค\"],\n    rows    = [\n        [\n            name,\n            f\"v{get_latest_version(name)}\",\n            \"Self-Consistency\" if \"sc_\" in name\n            else \"Prompt Chaining\" if \"chain_\" in name\n            else \"Self-Critique\",\n        ]\n        for name in all_prompt_names\n    ],\n    title = \"Prompt Registry Summary\",\n)\n\n# ── Links ──\nprint_divider(\"═\")\nprint(f\"\\n  🌐 ดูผลใน MLflow UI : {MLFLOW_URI}\")\nprint(f\"  📊 Experiment       : prompt_engineering_lab\")\nprint(f\"  📦 Prompts          : {len(all_prompt_names)} prompts registered\")\nprint_divider(\"═\")\n\n# %% [markdown]\n# ---\n# ## 📝 สรุปบทเรียน (Key Takeaways)\n#\n# ### 3 เทคนิค Prompt Engineering\n#\n# | เมื่อไหร่ควรใช้? | เทคนิค | เหตุผล |\n# |----------------|--------|--------|\n# | งาน Math / Logic / ต้องการความแม่นยำ | **Self-Consistency** | Majority vote ลดโอกาสผิด |\n# | งานซับซ้อนหลายขั้นตอน (เช่น วิเคราะห์ document) | **Prompt Chaining** | แยก focus, debug ง่าย |\n# | หัวข้อ controversial / ต้องการ unbiased answer | **Self-Critique** | วิพากษ์ตัวเอง → รอบด้านขึ้น |\n# | Prompt ยังให้ผลไม่ดีพอ | **Reflection Loop** | วิเคราะห์ข้อผิด → auto-improve |\n#\n# ### MLflow Best Practices\n#\n# | ✅ ควรทำ | ❌ ไม่ควรทำ |\n# |---------|-----------|\n# | ใส่ `commit_message` ทุกครั้งที่ register | Register โดยไม่มี message |\n# | ติดตาม version ด้วย `track_version()` | ใช้ version hardcode |\n# | `log_metric` ทุกครั้งที่ evaluate | วัดผลแค่ในหัว |\n# | ตั้งชื่อ prompt ให้สื่อความหมาย | ตั้งชื่อ prompt1, prompt2 |\n#\n# ### แนวคิดที่สำคัญที่สุด\n#\n# > **\"Prompt Engineering ไม่ใช่ศิลปะ แต่เป็นวิทยาศาสตร์\"**\n# >\n# > วัดผล → วิเคราะห์ → ปรับปรุง → วัดผลซ้ำ\n# > เหมือนการพัฒนา Software ที่ดี ต้องมี test และ iteration\n#\n# ---\n# **🎉 ยินดีด้วย! คุณเรียนจบ Lab นี้แล้ว**\n#\n# สิ่งที่คุณทำได้แล้ว:\n# - ✅ เขียนและทดสอบ 3 เทคนิค Prompt Engineering\n# - ✅ ลงทะเบียนและจัดการ Prompt versions ใน MLflow\n# - ✅ สร้าง Reflection Loop เพื่อปรับปรุง Prompt อัตโนมัติ\n# - ✅ ติดตามและเปรียบเทียบผลด้วย MLflow Tracking\n```"
    }
  ],
  "stop_reason": "end_turn",
  "usage": {
    "input_tokens": 11633,
    "cache_creation_input_tokens": 0,
    "cache_read_input_tokens": 0,
    "output_tokens": 23421
  }
}

In [9]:
# Simple extraction
text = response["content"][1]["text"]
print(text)

# Save to file
with open("output.txt", "w", encoding="utf-8") as f:
    f.write(text)

# %%
```python
# %% [markdown]
# # 🧪 Lab: Prompt Engineering กับ MLflow
# ## เรียนรู้ 3 เทคนิค + Reflection Loop + การติดตามผลด้วย MLflow
#
# ---
#
# ## 🎯 เป้าหมายของ Lab นี้
#
# ในการพัฒนา AI Application จริง เราต้องการ:
# 1. **เทคนิคการเขียน Prompt ที่ดี** → ให้ AI ตอบถูกต้องและมีคุณภาพ
# 2. **ระบบติดตามผล** → รู้ว่า Prompt เวอร์ชันไหนดีที่สุด
# 3. **การปรับปรุงอัตโนมัติ** → ให้ AI ช่วยปรับ Prompt เอง
#
# Lab นี้จะสอน 3 เทคนิคหลัก + วิธีใช้ MLflow ติดตามทุกอย่าง
#
# ---
#
# ## 🗺️ แผนผังรวม (Big Picture)
#
# ```
# ┌─────────────────────────────────────────────────────────────────┐
# │                    3 เทคนิค Prompt Engineering                  │
# │                                                                 │
# │  Technique 1: Self-Consistency                                  │
# │  ┌─────────────────────────────────────────────────────┐        │
# │  │  ถามคำถามเดิม 5 ครั้ง → เปรียบเทียบ → โหวตเสียงข้างมาก  │        │
# │  │  Problem → [Path1: 42] [Path2: 42] [Path3: 38] → 4